In [7]:
import pandas as pd  
import numpy as np 
from collections import Counter

In [25]:
data = pd.read_csv('bank.csv', sep = ';')

columns = ['job','education',  'marital', 'default', 'housing', 'loan']


In [26]:
from sklearn.feature_extraction import DictVectorizer
dictvectorizer=DictVectorizer()
pata=dictvectorizer.fit_transform(job)

pata

AttributeError: 'str' object has no attribute 'items'

In [20]:
def evaluate_classifier(predicted, observed):
    import pandas as pd 
    if(len(predicted) != len(observed)):
        print('\nevaluate_classifier error:',\
             ' predicted and observed must be the same length\n')
        return(None) 
    if(len(set(predicted)) != 2):
        print('\nevaluate_classifier error:',\
              ' predicted must be binary\n')
        return(None)          
    if(len(set(observed)) != 2):
        print('\nevaluate_classifier error:',\
              ' observed must be binary\n')
        return(None)          

    predicted_data = predicted
    observed_data = observed
    input_data = {'predicted': predicted_data,'observed':observed_data}
    input_data_frame = pd.DataFrame(input_data)
    
    cmat = pd.crosstab(input_data_frame['predicted'],\
        input_data_frame['observed']) 
    a = float(cmat.ix[0,0])
    b = float(cmat.ix[0,1])
    c = float(cmat.ix[1,0]) 
    d = float(cmat.ix[1,1])
    n = a + b + c + d
    predictive_accuracy = (a + d)/n
    true_positive_rate = a / (a + c)
    false_positive_rate = b / (b + d)
    precision = a / (a + b)
    specificity = 1 - false_positive_rate   
    expected_accuracy = (((a + b)*(a + c)) + ((b + d)*(c + d)))/(n * n)
    kappa = (predictive_accuracy - expected_accuracy)\
       /(1 - expected_accuracy)   
    return(a, b, c, d, predictive_accuracy, true_positive_rate, specificity,\
        false_positive_rate, precision, expected_accuracy, kappa)

In [14]:
import pandas as pd  # DataFrame structure and operations
import numpy as np  # arrays and numerical processing
import matplotlib.pyplot as plt  # 2D plotting
import statsmodels.api as sm  # logistic regression
import statsmodels.formula.api as smf  # R-like model specification
import patsy  # translate model specification into design matrices


D:\ANACONDA\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [15]:
bank = pd.read_csv('bank.csv', sep = ';')
print(bank.head())

# define jobtype variable
#�������������
job_to_jobtype = {'admin.':'White Collar',\
    'entrepreneur':'White Collar',\
    'management':'White Collar',\
    'self-employed':'White Collar',\
    'blue-collar':'Blue Collar',\
    'services':'Blue Collar',\
    'technician':'Blue Collar'}
bank['jobtype'] = bank['job'].map(job_to_jobtype)
bank['jobtype'] = bank['jobtype'].fillna('Other/Unknown')

# set marital variable
#�������״̬
marital_to_label = {'divorced':'Divorced',\
    'married':'Married',\
    'single':'Single'}
bank['marital'] = bank['marital'].map(marital_to_label)
bank['marital'] = bank['marital'].fillna('Unknown')

# set education variable
#����ˮƽ
education_to_label = {'primary':'Primary',\
    'secondary':'Secondary',\
    'tertiary':'Tertiary'}
bank['education'] = bank['education'].map(education_to_label)
bank['education'] = bank['education'].fillna('Unknown')

# set no/yes variable labels
#һЩ�߼�����
noyes_to_label = {'no':'No', 'yes':'Yes'}
bank['default'] = bank['default'].map(noyes_to_label)
bank['default'] = bank['default'].fillna('No')

bank['housing'] = bank['housing'].map(noyes_to_label)
bank['housing'] = bank['housing'].fillna('No')

bank['loan'] = bank['loan'].map(noyes_to_label)
bank['loan'] = bank['loan'].fillna('No')

# code response as binary variable
#���յ���Ӧ���
noyes_to_binary = {'no':0, 'yes':1}
bank['response'] = bank['response'].map(noyes_to_binary)
bank['response'] = bank['response'].fillna('No')

# work only with bank clients who are being approached for the first time  
filter = bank['pdays'].map(lambda d: d == -1)

# apply the filter and select columns needed for targeting model
bankwork = pd.DataFrame(bank[filter], columns = ['response','age','jobtype',\
    'education',  'marital', 'default', 'balance', 'housing', 'loan'])
print(bankwork.head()) 
print(bankwork.shape)   

# examine descriptive statistics and frequency tables for variables in model
print(bankwork.describe())
print('\njobtype:\n',bankwork['jobtype'].value_counts())
print('\nmarital:\n',bankwork['marital'].value_counts())
print('\neducation:\n',bankwork['education'].value_counts())
print('\ndefault:\n',bankwork['default'].value_counts())
print('\nhousing:\n',bankwork['housing'].value_counts())
print('\nloan:\n',bankwork['loan'].value_counts())

# examine means of continuous explanatory variables by response
print(bankwork.pivot_table(['age'], index = ['response']))
print(bankwork.pivot_table(['balance'], index = ['response']))

# baseline response rate computed (will be used later)
filter_took_offer = bankwork['response'].map(lambda d: d == 1)
baseline_response_rate = len(bankwork[filter_took_offer]) / len(bankwork)
print('\nBaseline proportion of clients responding to offer: ',\
    round(baseline_response_rate,5), '\n')

# examine proportion responding across levels 
# of categorical variables
print(bankwork.pivot_table(['response'], index = ['jobtype']))
print(bankwork.pivot_table(['response'], index = ['education']))
print(bankwork.pivot_table(['response'], index = ['marital']))
print(bankwork.pivot_table(['response'], index = ['default']))
print(bankwork.pivot_table(['response'], index = ['housing']))
print(bankwork.pivot_table(['response'], index = ['loan']))

# specify model for logisitc regression
bank_spec = 'response ~ age + jobtype + education + marital +\
    default + balance + housing + loan'

# ----------------------------------
# fit logistic regression model 
# ----------------------------------
# convert R-like formula into design matrix needed for statsmodels        
y,x = patsy.dmatrices(bank_spec, bankwork, return_type = 'dataframe')    

my_logit_model = sm.Logit(y,x)
# fit the model to the full data set
my_logit_model_fit = my_logit_model.fit()
# print(my_logit_model_fit.summary())

# predicted probability of reponding to the offer
bankwork['pred_logit_prob'] = my_logit_model_fit.predict(linear = False)

# map target from probability cutoff specified
def prob_to_pred(x, cutoff):
    if(x > cutoff):
        return(1)
    else:
        return(0)

# try cutoff set at 0.50
bankwork['pred_logit_50'] =\
    bankwork['pred_logit_prob'].\
    apply(lambda d: prob_to_pred(d, cutoff = 0.50))    
print('\nConfusion matrix for 0.50 cutoff\n',\
    pd.crosstab(bankwork.pred_logit_50, bankwork.response, margins = True))    
# cutoff 0.50 does not work for targeting... all predictions 0 or No    

# try cutoff set at 0.10
bankwork['pred_logit_10'] =\
    bankwork['pred_logit_prob'].\
    apply(lambda d: prob_to_pred(d, cutoff = 0.10))    
print('\nConfusion matrix for 0.10 cutoff\n',\
    pd.crosstab(bankwork.pred_logit_10, bankwork.response, margins = True)) 

print('\n Logistic Regression Performance (0.10 cutoff)\n',\
    'Percentage of Targets Correctly Classified:',\
    100 * round(evaluate_classifier(bankwork['pred_logit_10'],\
    bankwork['response'])[4], 3),'\n')

# direct calculation of lift 
# decile labels from highest to lowest 
decile_label = []
for i in range(10):
    decile_label.append('Decile_'+str(10 - i))
# draws on baseline response rate computed earlier    
def lift(x):
    return(x / baseline_response_rate)

prediction_deciles = pd.qcut(bankwork.pred_logit_prob, 10, labels = decile_label)
decile_groups = bankwork.response.groupby(prediction_deciles)
print(decile_groups.mean())
lift_values = decile_groups.mean() / baseline_response_rate
print('\nLift Chart Values by Decile:\n', lift_values, '\n')

   age          job  marital  education default  balance housing loan  \
0   30   unemployed  married    primary      no     1787      no   no   
1   33     services  married  secondary      no     4789     yes  yes   
2   35   management   single   tertiary      no     1350     yes   no   
3   30   management  married   tertiary      no     1476     yes  yes   
4   59  blue-collar  married  secondary      no        0     yes   no   

    contact  day month  duration  campaign  pdays  previous poutcome response  
0  cellular   19   oct        79         1     -1         0  unknown       no  
1  cellular   11   may       220         1    339         4  failure       no  
2  cellular   16   apr       185         1    330         1  failure       no  
3   unknown    3   jun       199         4     -1         0  unknown       no  
4   unknown    5   may       226         1     -1         0  unknown       no  
   response  age        jobtype  education  marital default  balance housing  \
0

D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
